In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import random

In [2]:
coupon_list_train = pd.read_csv('input/coupon_list_train.csv')
coupon_list_test = pd.read_csv('input/coupon_list_test.csv')
user_list = pd.read_csv('input/user_list.csv')
coupon_purchases_train = pd.read_csv("input/coupon_detail_train.csv")

coupon_area_train = pd.read_csv('input/coupon_area_train.csv')
coupon_area_test = pd.read_csv('input/coupon_area_test.csv')
coupon_visit_train = pd.read_csv('input/coupon_visit_train.csv')

In [3]:
purchased_coupons_train = coupon_purchases_train.merge(coupon_list_train, on='COUPON_ID_hash', how='inner')

In [4]:
purchased_coupons_train.head(10)

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,USER_ID_hash,COUPON_ID_hash,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,...,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,1,2012-03-28 15:06:06,兵庫,c820a8882374a4e472f0984a8825893f,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
1,1,2012-03-26 23:26:24,千葉,028ac37f96a17e6be2db51ed0884ac5a,dd8a79645e8170db6a91bad1d89b4055,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
2,1,2012-03-26 13:50:20,新宿・高田馬場・中野・吉祥寺,eb7f867c9f1b14c1a2b1cc47004828ae,658249379aaaf2b2320061e5218c0c2c,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
3,1,2012-03-28 22:46:58,和歌山,fddda2bdaba87fc9b86d97e908152014,1c4aad19f033d97dc163b481116de322,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
4,1,2012-03-29 15:09:51,宮崎,6659c8fdb5f66b4013613ff770f91db7,e78ae0887c4a9dce91ff0d2a45a9f72c,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
5,1,2012-03-29 12:42:37,千葉,be5af03efe7e4bc2f1e7e8317531cd74,39cabe60f73ea8cad96d7f895ea944ee,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
6,1,2012-03-28 22:45:16,和歌山,aae8c29627af3489c530f141eb02a3a1,fca1e37548c9ca5415a9850a1b9b2c28,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
7,1,2012-03-29 18:10:28,栃木,ea4cf60407a03246be6c588c12542780,3511c63ae9ce45fc3b5cf5054956d5a4,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
8,1,2012-03-28 00:05:32,栃木,448cec2c62179725c9ac0d93006074d0,77b92939627690028e22834d7920bf4e,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
9,2,2012-03-27 11:47:59,栃木,f8f426b0f2beb05978e856a727febac8,1ce49ceb6f55d3b3ed3180f811ec3ffb,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺


In [5]:
features = ['COUPON_ID_hash', 'USER_ID_hash',
            'GENRE_NAME', 'DISCOUNT_PRICE', 'PRICE_RATE',
            'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
            'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
            'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']

In [6]:
purchased_coupons_train = purchased_coupons_train[features]

In [7]:
purchased_coupons_train.shape[0]

168996

In [8]:
df = purchased_coupons_train.copy()
df = df[:10000] 

In [9]:
df['DISCOUNT_PRICE'] = 1 / np.log10(df['DISCOUNT_PRICE'])
df['PRICE_RATE'] = (df['PRICE_RATE'] / 100) ** 2
# features.extend(['DISCOUNT_PRICE', 'PRICE_RATE'])

/home/nunoferreira/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


In [10]:
# categoricals = ['GENRE_NAME', 'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED',
#                 'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN',
#                 'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']
# df_categoricals = df[categoricals]
# df_categoricals = pd.get_dummies(df_categoricals, dummy_na=False)

In [11]:
# continuous = list(set(features) - set(categoricals))
# df = pd.concat([df[continuous], df_categoricals], axis=1)

In [12]:
# NAN_SUBSTITUTION_VALUE = 1
# df = df.fillna(NAN_SUBSTITUTION_VALUE)

In [13]:
df_temp = df.copy()
# df_temp = df_temp.drop(['COUPON_ID_hash', 'USER_ID_hash'], axis=1)

categorical_cols = ['COUPON_ID_hash', 'USER_ID_hash', 'GENRE_NAME', 'large_area_name', 'ken_name', 'small_area_name']

In [14]:
df_temp

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,34c48f84026e08355dc3bd19b427f09a,d9dca3cb44bab12ba313eaa681f663eb,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
1,34c48f84026e08355dc3bd19b427f09a,dd8a79645e8170db6a91bad1d89b4055,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
2,34c48f84026e08355dc3bd19b427f09a,658249379aaaf2b2320061e5218c0c2c,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
3,34c48f84026e08355dc3bd19b427f09a,1c4aad19f033d97dc163b481116de322,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
4,34c48f84026e08355dc3bd19b427f09a,e78ae0887c4a9dce91ff0d2a45a9f72c,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
5,34c48f84026e08355dc3bd19b427f09a,39cabe60f73ea8cad96d7f895ea944ee,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
6,34c48f84026e08355dc3bd19b427f09a,fca1e37548c9ca5415a9850a1b9b2c28,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
7,34c48f84026e08355dc3bd19b427f09a,3511c63ae9ce45fc3b5cf5054956d5a4,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
8,34c48f84026e08355dc3bd19b427f09a,77b92939627690028e22834d7920bf4e,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
9,34c48f84026e08355dc3bd19b427f09a,1ce49ceb6f55d3b3ed3180f811ec3ffb,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺


In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_temp[categorical_cols] = df_temp[categorical_cols].apply(lambda col: le.fit_transform(col))
df_temp = df_temp.fillna(1)
df_temp

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,12,6643,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
1,12,6761,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
2,12,3072,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
3,12,842,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
4,12,7053,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
5,12,1730,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
6,12,7669,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
7,12,1604,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
8,12,3617,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
9,12,864,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9


In [25]:
l = df_temp.shape[0]
f = len(df_temp.columns)
t = AnnoyIndex(f, 'angular')

In [26]:
for i in range(l):
    v = [df_temp.values[i][z] for z in range(f)]
    t.add_item(i, v)

In [27]:
t.build(50)

True

In [56]:
#sim = t.get_nns_by_item(0, 5, include_distances=True)
sim = t.get_nns_by_item(0, 5)
sim_33 = t.get_nns_by_item(33, 5)
sim_60 = t.get_nns_by_item(60, 5)
sim_22 = t.get_nns_by_item(22, 5)
sim_1 = t.get_nns_by_item(1, 5)

i = list(set(sim_33) & set(sim_60))
i2 = list(set(sim_22) & set(sim_60))
i3 = list(set(sim_22) & set(sim_1))
i2, i, i3, i0


([0, 44], [0, 44], [0, 44], [25, 60, 1])

In [55]:
df_temp.loc[i2]

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,12,6643,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
44,12,6427,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9


In [46]:
df_temp.loc[sim]

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
25,12,6393,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
33,12,6428,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
60,12,6463,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
22,12,6604,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
1,12,6761,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9


In [48]:
df_temp.loc[sim_33]

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
25,12,6393,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
44,12,6427,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
60,12,6463,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
0,12,6643,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
1,12,6761,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9


In [58]:
sim_0 = t.get_nns_by_item(0, 5)
sim_44 = t.get_nns_by_item(44, 5)

In [62]:
df_temp.loc[[0, 44]]

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,12,6643,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
44,12,6427,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9


In [59]:
df_temp.loc[sim_0]

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
25,12,6393,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
33,12,6428,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
60,12,6463,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
22,12,6604,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
1,12,6761,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9


In [60]:
df_temp.loc[sim_44]

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
25,12,6393,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
33,12,6428,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
60,12,6463,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
22,12,6604,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
1,12,6761,8,0.312766,0.6084,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,9
